# Settings

## modules

In [1]:
%load_ext autoreload
%autoreload 2

In [32]:
import parse_lumpy
import parse_destruct
import consensus

In [21]:
import copy
import numpy as np
import pandas as pd
from collections import defaultdict
import pandas as pd
from intervaltree import IntervalTree
from wgs.utils import csvutils
import vcf

# Data

In [5]:
lumpy_path = '../data/breakpoints/HCM-BROD-0011-C71-01A_lumpy.vcf'
destruct_path = '../data/breakpoints/HCM-BROD-0011-C71-01A_destruct_breakpoints.tsv'

# Proc

## configs

In [6]:
config = {
    'chromosomes': list(map(str, range(1, 23))) + ['X'],
    'lumpy_paths': {
        'extractSplitReads_BwaMem': 'lumpy_extractSplitReads_BwaMem',
        'samtools': 'samtools',
        'lumpyexpress': 'lumpyexpress',
    },
    'parse_lumpy': {
        'deletion_size_threshold': 0,
        'tumour_read_support_threshold': 0,
    },
    'parse_destruct': {
        'deletion_size_threshold': 1000,
        'foldback_threshold': 30000,
        'readsupport_threshold': 4,
        'breakdistance_threshold': 30
    },
    'consensus': {
        'confidence_interval_size': 500,
    },
}

## process lumpy

In [28]:
lumpy_output_path = '../data/breakpoints/HCM-BROD-0011-C71-01A_filtered_lumpy.csv'
vcfdata = parse_lumpy.parse_vcf(lumpy_path)
vcfdata = parse_lumpy.parse_vcf_group(vcfdata)
vcfdata = parse_lumpy.create_data(vcfdata)
vcfdata = parse_lumpy.convert_to_df(vcfdata)
vcfdata = parse_lumpy.filter_calls(vcfdata, config['parse_lumpy'])
parse_lumpy.write(lumpy_output_path, vcfdata)

## process destruct

In [29]:
destruct_output_path = '../data/breakpoints/HCM-BROD-0011-C71-01A_filtered_destruct.csv'
parse_destruct.parser(destruct_path, destruct_output_path, config['parse_destruct'], foldback_threshold=config['parse_destruct']['foldback_threshold'])

/rtsess01/compute/juno/shah/users/chois7/tickets/akansha-pipeline-qc/notebooks/parse_destruct.py:17: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['break_distance'][df['chromosome_1'] != df['chromosome_2']] = np.inf
/rtsess01/compute/juno

## consensus

In [39]:
consensus_path = '../data/breakpoints/consensus.csv'
consensus.consensus(
    destruct_output_path, lumpy_output_path, consensus_path, confidence_interval=config['consensus']['confidence_interval_size']
)